In [1]:
import numpy as np
from numba import njit, float64, int64
import time

## Naive Python code

In [2]:
def est_null(mu1, mu2, n, iteration = 10000):
    np.random.seed(42)
    
    x = np.random.normal(loc = mu1, size = n)
    y = np.random.normal(loc = mu2, size = n)
    
    null_dist = np.zeros(iteration)
    xy = np.r_[x,y]
    
    for i in range(iteration):
        temp = np.random.permutation(xy)
        temp_x, temp_y = temp[:n] , temp[n:]
        null_dist[i] = np.mean(temp_x) - np.mean(temp_y)
    
    return null_dist

## Numba Python code

In [4]:
@njit('float64[:](float64[:],float64[:],float64[:],int64)')
def njit_est_null(x, y, xy, iteration = 10000):
    n = x.shape[0]
    null_dist = np.zeros(iteration)
    
    np.random.seed(42)
    for i in range(iteration):
        temp = np.random.permutation(xy)
        temp_x, temp_y = temp[:n], temp[n:]
        null_dist[i] = np.mean(temp_x) - np.mean(temp_y)
        
    return null_dist

## Compute p-value

In [5]:
def compute_pvalue(mu1, mu2, n, null_dist, iteration = 10000):
    
    pvalue_vec = np.zeros(iteration)
    
    for i in range(iteration):
        x = np.random.normal(loc = mu1, size = n)
        y = np.random.normal(loc = mu2, size = n)
    
        diff_value = np.mean(x) - np.mean(y)
        pvalue_vec[i] = 1.0 - np.mean(np.abs(diff_value) > np.abs(null_dist))
        
    return pvalue_vec

## Setting 1: $\mu_1 = 0, \mu_2 = 0$ length = 50

### Time for naive_python_code

In [6]:
naive_time_1 = []
np.random.seed(42)
for i in range(10):
    
    t1 = time.time()
    est_null(mu1 = 0, mu2 = 0, n=50)
    t2 = time.time()
    
    naive_time_1.append(t2-t1)

In [7]:
print(np.mean(naive_time_1))
print(np.std(naive_time_1)/np.sqrt(10))

0.3145370244979858
0.007614325636712737


### Time for nopython mode of just-in-time compilation 

In [8]:
njit_time_1 = []
np.random.seed(42)
for i in range(10):
    t1 = time.time()
    x = np.random.normal(loc=0, size = 50)
    y = np.random.normal(loc=0, size = 50)
    xy = np.r_[x,y]
    njit_est_null(x,y,xy,iteration=10000)
    t2 = time.time()
    
    njit_time_1.append(t2-t1)

In [9]:
print(njit_time_1[0])
print(np.mean(njit_time_1[1:]))
print(np.std(njit_time_1[1:])/np.sqrt(9))

0.022687435150146484
0.018136686748928495
0.0007541533357192311


### Compute p-value given by computed null distribution

In [11]:
np.random.seed(42)
x = np.random.normal(loc=0, size = 50)
y = np.random.normal(loc=0, size = 50)
xy = np.r_[x,y]
null_dist_1 = njit_est_null(x,y,xy,iteration = 10000)

In [12]:
pvalue_vec1=compute_pvalue(mu1 = 0, mu2 = 0, n =50, null_dist = null_dist_1)

#size 
size_power1= np.mean(pvalue_vec1 < 0.05) 

## Setting 2: $\mu_1 = 0, \mu_2 = 0.5$ length = 100

### Time for naive_python_code

In [13]:
naive_time_2 = []
np.random.seed(42)
for i in range(10):
    
    t1 = time.time()
    est_null(mu1 = 0, mu2 = 0.5, n=100)
    t2 = time.time()
    
    naive_time_2.append(t2-t1)

In [14]:
print(np.mean(naive_time_2))
print(np.std(naive_time_2)/np.sqrt(10))

0.3664196252822876
0.027012921819173575


### Time for nopython mode of just-in-time compilation 

In [19]:
## Redefine the njit function to recompile
@njit('float64[:](float64[:],float64[:],float64[:],int64)')
def njit_est_null(x, y, xy, iteration = 10000):
    n = x.shape[0]
    null_dist = np.zeros(iteration)
    
    np.random.seed(42)
    for i in range(iteration):
        temp = np.random.permutation(xy)
        temp_x, temp_y = temp[:n], temp[n:]
        null_dist[i] = np.mean(temp_x) - np.mean(temp_y)
        
    return null_dist

In [20]:
njit_time_2 = []
np.random.seed(42)
for i in range(10):
    t1 = time.time()
    x = np.random.normal(loc=0, size = 100)
    y = np.random.normal(loc=0.5, size = 100)
    xy = np.r_[x,y]
    njit_est_null(x,y,xy,iteration=10000)
    t2 = time.time()
    
    njit_time_2.append(t2-t1)

In [21]:
print(njit_time_2[0])
print(np.mean(njit_time_2[1:]))
print(np.std(njit_time_2[1:])/np.sqrt(9))

0.035750389099121094
0.03679770893520779
0.002996340052900353


### Compute p-value given by computed null distribution

In [22]:
np.random.seed(42)
x = np.random.normal(loc=0, size = 100)
y = np.random.normal(loc=0.5, size = 100)
xy = np.r_[x,y]
null_dist_2 = njit_est_null(x,y,xy,iteration = 10000)

In [25]:
pvalue_vec2=compute_pvalue(mu1 = 0, mu2 = 0.5, n =100, null_dist = null_dist_2)

#power 
size_power2= np.mean(pvalue_vec2 < 0.05) 

In [26]:
size_power2

0.9495